In [150]:
import re
import pickle
from copy import deepcopy
from typing import Optional, List, Union
from pprint import pprint

import pandas as pd

In [28]:
LOG_FILE_PATH = 'access.log.1'
LOG_PICKLE_PATH = 'access.log.1.pckl'

In [123]:
log_0 = '10.11.1.9 2242.lnsigo.mipt.ru - - [17/Dec/2018:06:17:16 +0300] "GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd1/getUpdates HTTP/1.1" 200 26 "-" "python-requests/2.16.5" "-"'
log_1 = '10.11.1.9 2242.lnsigo.mipt.ru - [17/Dec/2018:06:17:16 +0300] "GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd2/getUpdates HTTP/1.1" 200 26 "-" "python-requests/2.16.5" "-"'
log_2 = '93.158.156.97 2246.lnsigo.mipt.ru - [17/Dec/2018:06:17:16 +0300] "POST / HTTP/1.1" 502 568 "-" "YaAlice/1.0.0.0 Chrome/19.0.1084.5409 Safari/536.5" "-"'
log_3 = '10.11.1.9 2242.lnsigo.mipt.ru - [17/Dec/2018:06:17:16 +0300] "GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd3/getUpdates HTTP/1.1" 200 26 "-" "python-requests/2.16.5" "-"'
log_example = f'{log_0}\n{log_1}\n{log_2}\n{log_3}'

In [65]:
def load_logs(log_file_path: str, row_num: Optional[int] = None, parser: Optional[callable] = None) -> pd.DataFrame:
    with open(log_file_path, 'r') as f:
        log = f.readlines() if not row_num or row_num < 1 else [f.readline() for _ in range(row_num)]
    log_df = pd.DataFrame(data=log, columns=['raw'])
    return(log_df)

In [121]:
#pattern_main = '^|\n(\S+?)\s(\S+?)\s(\S+?)\s(\[.+?\])\s(".+?")\s(.+?)\s(.+?)\s(".+?")\s(".+?")\s(".+?")\n|$'
#pattern_main = '(\S+?)\s(\S+?)\s(\S+?)\s(\[.+?\])\s(".+?")\s(.+?)\s(.+?)\s(".+?")\s(".+?")\s(".+?")'
pattern_main = '^(\S+?)\s(\S+?)\s(\S+?)\s(\[.+?\])\s(".+?")\s(.+?)\s(.+?)\s(".+?")\s(".+?")\s(".+?")$'

In [148]:
class LogDataFrame:
    def __init__(self):
        self._reserved = ['raw', '_regex', '_columns']
        self._config = {'raw': None}
        self._df = pd.DataFrame(columns=['raw']) #: Optional[pd.DataFrame] = None
        self._df_unrec = pd.DataFrame(columns=['raw']) #: Optional[pd.DataFrame] = None
            
    def __call__(self) -> pd.DataFrame:
        pass
    
    def _apply_config(self, log_df: pd.DataFrame) -> pd.DataFrame:
        return log_df
    
    def add(self, add_data: Union[str, list, pd.DataFrame], column_name: Optional[str]: None) -> bool:
        add_data = deepcopy(add_data)
        if isinstance(add_data, str):
            add_data = add_data.strip('\n')
            add_data = re.sub('\n+', '\n', add_data)
            add_data = add_data.split('\n')
            add_data = pd.DataFrame(data={'raw': add_data})
        elif isinstance(add_data, list):
            add_data = pd.DataFrame(data={'raw': add_data})
        elif isinstance(add_data, pd.DataFrame):
            if not column_name or column_name not in add_data.columns:
                raise ValueError(f'Name {str(column_name)} was not set or found in input dataframe indexes')
            else:
                drop_labels = [column for column in add_data.columns if column != column_name]
                add_data.drop(labels=drop_labels, axis=1, inplace=True)
        else:
            raise TypeError(f'Wrong type of input data: {type(add_data)}')
        
        self._df = add_data
        print('rawk' in self._df.columns)
        print(self._df)
        
        
    #with open(log_file_path, 'r') as f:
    #    log = f.readlines()  
    #log = log_example
    

In [149]:
log_df = LogDataFrame()
log_df.add(['a', 'b', 'c'])

False
  raw
0   a
1   b
2   c


In [124]:
found = re.findall(pattern_main, log_example, flags = re.MULTILINE)
pprint(found)

[('10.11.1.9',
  '2242.lnsigo.mipt.ru',
  '-',
  '[17/Dec/2018:06:17:16 +0300]',
  '"GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd2/getUpdates HTTP/1.1"',
  '200',
  '26',
  '"-"',
  '"python-requests/2.16.5"',
  '"-"'),
 ('93.158.156.97',
  '2246.lnsigo.mipt.ru',
  '-',
  '[17/Dec/2018:06:17:16 +0300]',
  '"POST / HTTP/1.1"',
  '502',
  '568',
  '"-"',
  '"YaAlice/1.0.0.0 Chrome/19.0.1084.5409 Safari/536.5"',
  '"-"'),
 ('10.11.1.9',
  '2242.lnsigo.mipt.ru',
  '-',
  '[17/Dec/2018:06:17:16 +0300]',
  '"GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd3/getUpdates HTTP/1.1"',
  '200',
  '26',
  '"-"',
  '"python-requests/2.16.5"',
  '"-"')]


In [66]:
log: pd.DataFrame = load_logs(log_file_path=LOG_FILE_PATH, row_num=2)

In [67]:
print(log.head(5))

                                                 raw
0  10.11.1.9 2242.lnsigo.mipt.ru - [16/Dec/2018:0...
1  10.11.1.9 2242.lnsigo.mipt.ru - [16/Dec/2018:0...


In [68]:
log_str = log.iat[0, 0]
print(log_str)

10.11.1.9 2242.lnsigo.mipt.ru - [16/Dec/2018:06:25:12 +0300] "GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd/getUpdates HTTP/1.1" 200 26 "-" "python-requests/2.16.5" "-"



In [69]:
pattern = r'^(\S+?)\s(\S+?)\s(\S+?)\s(\[.+?\])\s(".+?")\s(.+?)\s(.+?)\s(".+?")\s(".+?")\s(".+?")$'
match = re.search(pattern, log_str)
if match:
    print(match.groups())

('10.11.1.9', '2242.lnsigo.mipt.ru', '-', '[16/Dec/2018:06:25:12 +0300]', '"GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd/getUpdates HTTP/1.1"', '200', '26', '"-"', '"python-requests/2.16.5"', '"-"')
